# Cirq - t|ket> routing example

First import cirq and required classes and functions from t|ket>.
SquareGrid is the t|ket> class representing a rectangular grid of qubits, with qubits indexed continuously from (0,0), e.g. for a 3x3 grid, the indexing is: 

0 1 2 <br>
3 4 5 <br>
6 7 8

`route_circuit` will take our circuit and return a routed one. Routing involves optimally adding swap commands such that the necessary 2-qubit interactions only take place between nearest-neighbour qubits (as is normally the case with superconducting qubits). <br>
`get_grid_qubits` is a helper function to return a list of cirq.GridQubit which correspond to a list of provided indices on the given SquareGrid.

In [1]:
import cirq
from pytket import SquareGrid, route_circuit
from pytket.cirq_convert import get_grid_qubits

Let's initialize a 3x3 grid and get the corresponding 9 qubits.

In [2]:
arc = SquareGrid(3, 3)
qubits = get_grid_qubits(arc, range(9))

Generate an arbitrary sample circuit. See the [readme](https://github.com/CQCL/pytket/blob/master/README.md) for currently supported gates, more added soon!

In [3]:
circuit = cirq.Circuit.from_ops(
        cirq.H(qubits[0]),
        cirq.X(qubits[1]),
        cirq.CNOT(qubits[0], qubits[4]),
        cirq.Y(qubits[4]),
        cirq.CNOT(qubits[2], qubits[4]),
        cirq.CNOT(qubits[3], qubits[4]),
        cirq.Y(qubits[4]),
        cirq.CNOT(qubits[3], qubits[5]),
        cirq.Z(qubits[4])
    )
circuit

(0, 0): ───H───@───────────────────────────
               │
(0, 1): ───X───┼───────────────────────────
               │
(0, 2): ───────┼───────@───────────────────
               │       │
(1, 0): ───────┼───────┼───@───────@───────
               │       │   │       │
(1, 1): ───────X───Y───X───X───Y───┼───Z───
                                   │
(1, 2): ───────────────────────────X───────

And now we can route the circuit on the architecture, subject to the constraint that the 2-qubit interactions can only occur between neighbouring qubits. 

In [4]:
route_circuit(circuit, arc)

(0, 0): ───H───×───────────────────────────────────
               │
(0, 1): ───X───×───@───×───@───────────────────────
                   │   │   │
(0, 2): ───────────┼───×───┼───────────────────────
                   │       │
(1, 0): ───────────┼───────┼───@───────────×───────
                   │       │   │           │
(1, 1): ───────────X───Y───X───X───Y───Z───×───@───
                                               │
(1, 2): ───────────────────────────────────────X───

Three swaps were needed to route this circuit.